In [44]:
#install libraries

_ = !pip install -r ../requirements.txt

In [45]:
# notebook to get information about dolar price in Argentina and save it in a csv file

import requests


# In[2]:



In [58]:
dolar_url = 'https://api.bluelytics.com.ar/v2/latest'

dolar = requests.get(dolar_url)
print(dolar)

<Response [200]>


In [59]:
#parseo el json
dolar_json = dolar.json()
print(dolar_json)

{'oficial': {'value_avg': 229.5, 'value_sell': 234.0, 'value_buy': 225.0}, 'blue': {'value_avg': 465.5, 'value_sell': 468.0, 'value_buy': 463.0}, 'oficial_euro': {'value_avg': 246.0, 'value_sell': 251.0, 'value_buy': 241.0}, 'blue_euro': {'value_avg': 500.5, 'value_sell': 503.0, 'value_buy': 498.0}, 'last_update': '2023-05-04T15:40:26.938401-03:00'}


In [60]:
#create a series with the json to put as a row in the dataframe
dolar_row = {}

for i in dolar_json:
    if i == 'last_update':
        dolar_row["last_update"] = dolar_json[i]
    else:
        for j in dolar_json[i]:
            dolar_row[f"dolar_{i}_{j}"] = dolar_json[i][j]

print(dolar_row)


{'dolar_oficial_value_avg': 229.5, 'dolar_oficial_value_sell': 234.0, 'dolar_oficial_value_buy': 225.0, 'dolar_blue_value_avg': 465.5, 'dolar_blue_value_sell': 468.0, 'dolar_blue_value_buy': 463.0, 'dolar_oficial_euro_value_avg': 246.0, 'dolar_oficial_euro_value_sell': 251.0, 'dolar_oficial_euro_value_buy': 241.0, 'dolar_blue_euro_value_avg': 500.5, 'dolar_blue_euro_value_sell': 503.0, 'dolar_blue_euro_value_buy': 498.0, 'last_update': '2023-05-04T15:40:26.938401-03:00'}


In [61]:
#convert dolar_row to a dataframe
dolar_df = pd.DataFrame(dolar_row, index=[0])
series = dolar_df.iloc[0]
print(dolar_df)

   dolar_oficial_value_avg  dolar_oficial_value_sell  dolar_oficial_value_buy  \
0                    229.5                     234.0                    225.0   

   dolar_blue_value_avg  dolar_blue_value_sell  dolar_blue_value_buy  \
0                 465.5                  468.0                 463.0   

   dolar_oficial_euro_value_avg  dolar_oficial_euro_value_sell  \
0                         246.0                          251.0   

   dolar_oficial_euro_value_buy  dolar_blue_euro_value_avg  \
0                         241.0                      500.5   

   dolar_blue_euro_value_sell  dolar_blue_euro_value_buy  \
0                       503.0                      498.0   

                        last_update  
0  2023-05-04T15:40:26.938401-03:00  


In [42]:

#save df to csv
dolar_df.to_csv('../src/data/dolar.csv', index=False)



In [62]:
import psycopg2

conn = psycopg2.connect(
    host="192.168.0.47",
    database="assets",
    user="postgres",
    password="postgres",
    port=5431)

cur = conn.cursor()



In [65]:
#query to insert the data in the database

#define a query to create a table if it doesn't exist
create_table_query = '''CREATE TABLE IF NOT EXISTS dollar
            (id SERIAL PRIMARY KEY,
            last_update timestamp,
            dolar_oficial_value_avg float,
            dolar_oficial_value_sell float,
            dolar_oficial_value_buy float,
            dolar_blue_value_avg float,
            dolar_blue_value_sell float,
            dolar_blue_value_buy float,
            dolar_oficial_euro_value_avg float,
            dolar_oficial_euro_value_sell float,
            dolar_oficial_euro_value_buy float,
            dolar_blue_euro_value_avg float,
            dolar_blue_euro_value_sell float,
            dolar_blue_euro_value_buy float); '''
            

#execute the query
cur.execute(create_table_query)
conn.commit()


In [63]:
series = dolar_df.iloc[0]

#query to insert the series in the table dolar
query = """INSERT 
            INTO dollar(dolar_oficial_value_avg, dolar_oficial_value_sell, dolar_oficial_value_buy, dolar_blue_value_avg, dolar_blue_value_sell, dolar_blue_value_buy, dolar_oficial_euro_value_avg, dolar_oficial_euro_value_sell, dolar_oficial_euro_value_buy, dolar_blue_euro_value_avg, dolar_blue_euro_value_sell, dolar_blue_euro_value_buy,last_update)
            VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s);"""

#execute the query
cur.execute(query, series)
conn.commit()

    

In [64]:
#query to read the table
query = "SELECT * FROM dolar"

#execute the query
cur.execute(query)
conn.commit()

#fetch the data
data = cur.fetchall()

#convert the data to a dataframe
dolar_df = pd.DataFrame(data, columns=['id','dolar_oficial_value_avg', 'dolar_oficial_value_sell', 'dolar_oficial_value_buy', 'dolar_blue_value_avg', 'dolar_blue_value_sell', 'dolar_blue_value_buy', 'dolar_oficial_euro_value_avg', 'dolar_oficial_euro_value_sell', 'dolar_oficial_euro_value_buy', 'dolar_blue_euro_value_avg', 'dolar_blue_euro_value_sell', 'dolar_blue_euro_value_buy','last_update'])



#print data
print(dolar_df)

   id    dolar_oficial_value_avg  dolar_oficial_value_sell  \
0   1 2023-05-04 14:55:26.357664                     229.5   
1   2 2023-05-04 15:40:26.938401                     229.5   

   dolar_oficial_value_buy  dolar_blue_value_avg  dolar_blue_value_sell  \
0                    234.0                 225.0                  464.5   
1                    234.0                 225.0                  465.5   

   dolar_blue_value_buy  dolar_oficial_euro_value_avg  \
0                 467.0                         462.0   
1                 468.0                         463.0   

   dolar_oficial_euro_value_sell  dolar_oficial_euro_value_buy  \
0                          246.0                         251.0   
1                          246.0                         251.0   

   dolar_blue_euro_value_avg  dolar_blue_euro_value_sell  \
0                      241.0                       499.5   
1                      241.0                       500.5   

   dolar_blue_euro_value_buy  last_

In [ ]:
#close the connection
cur.close()
conn.close()